In [62]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,mean_absolute_percentage_error
from sklearn.linear_model import SGDRegressor,LinearRegression,RANSACRegressor

In [63]:
with open('HomeDatasetTehran_R4_1404_02.txt', 'r', encoding='utf-8') as file:
    data = file.readlines()
df = pd.DataFrame([line.strip().split('\t') for line in data]).drop_duplicates()
df.columns = df.iloc[0]
df = df[1:]
df.head()

,h_type,date,adress,loc,num_floor,unit_per_floor,price,age,statuse,view,...,parking,warehouse,balcony,equipment,nearby,None,None,None,None,None
1,آپارتمان,1404/01/28,تهرانپارس خ 188غربی,4,5,2,"14,000,000,000 تومان",15,مسکونی,سنگ,...,1,1,1,شوفاژ_کولر_آسانسور_گاز,15,مسکونی,سنگ,شخصي,None,None
2,آپارتمان,1404/01/26,تهرانپارس بلوار پروین خ 196 شرقی,4,4,1,"4,500,000,000 تومان",1,تخلیه,سنگ,...,0,1,1,پکیج,1,تخلیه,سنگ,قولنامه اي,None,None
3,آپارتمان,1404/01/26,فرجام غربی خ حیدرخانی,4,5,1,"10,500,000,000 تومان",4,تخلیه,سنگ,...,1,1,1,کولر_آسانسور_پکیج,4,تخلیه,سنگ,شخصي,None,None
4,آپارتمان,1404/01/31,هروی گلستان,4,5,2,"18,000,000,000 تومان",17,مسکونی,سنگ,...,1,1,0,شوفاژ_کولر_آسانسور,17,مسکونی,سنگ,شخصي,None,None
5,آپارتمان,1404/01/26,هنگام چهارراه استقلال,4,4,2,"5,000,000,000 تومان",17,مسکونی,سنگ,...,1,1,0,کولر_گاز,17,مسکونی,سنگ,شخصي,None,None


In [64]:
columns_list = df.columns.tolist()
print(columns_list)

['h_type', 'date', 'adress', 'loc', 'num_floor', 'unit_per_floor', 'price', 'age', 'statuse', 'view', 'doc_status', 'north', 'sought', 'west', 'east', 'floor', 'area', 'num_sleep', 'tel', 'kitch', 'service', 'floor_covering', 'open', 'parking', 'warehouse', 'balcony', 'equipment', 'nearby', None, None, None, None, None]


check None columns:

In [65]:
print(df.iloc[:, 30].value_counts(),df.doc_status.value_counts())

شخصي           794
قولنامه اي      77
تعاوني          34
سنگ             19
دردست اقدام     15
مسکوني          12
منگوله دار       7
مسکونی           5
مشاع             3
اوقافي           2
آجر سه سانت      2
تخلیه            2
سيمان            1
آماده محضر       1
گرانيت           1
Name: count, dtype: int64 doc_status
شخصي            794
قولنامه اي       77
تعاوني           34
سنگ              19
دردست اقدام      15
مسکوني           12
منگوله دار        7
مسکونی            5
مشاع              3
آجر سه سانت       2
اوقافي            2
تخلیه             2
باز سازی شده      1
2 پاركینگ         1
12 متر بالكن      1
سيمان             1
آماده محضر        1
                  1
گرانيت            1
Name: count, dtype: int64


column 30 ,29,28 are duplicated

check 5th None column

In [66]:
filtered_df = df[df.iloc[:, 32].notna()]
df.drop(index=filtered_df.index,inplace=True)
filtered_df.iloc[:,7:]

,age,statuse,view,doc_status,north,sought,west,east,floor,area,...,parking,warehouse,balcony,equipment,nearby,None,None,None,None,None
36,400,13,30,مسکونی,0,1,0,1,1,150,...,1,1,0,شوفاژ_کولر_گاز,400,13,30,مسکونی,سنگ,شخصي
56,150,8,12,مسکونی,0,1,0,0,كلی,380,...,1,0,1,شوفاژ_کولر_گاز,150,8,12,مسکونی,سنگ,شخصي
174,1330,33,30,تخلیه,0,0,1,1,1,128,...,1,1,1,شوفاژ_کولر,1330,33,30,تخلیه,سنگ,شخصي
268,300,12,27,مسکونی,0,1,0,0,4,84,...,1,1,0,کولر_گاز,300,12,27,مسکونی,سنگ,شخصي
935,420,14,30,مسکونی,1,0,0,0,4,100,...,1,1,0,شوفاژ_کولر,420,14,30,مسکونی,تراورتن,شخصي
940,71,6,30,تخلیه,0,1,0,0,كلی,222,...,0,1,1,کولر_گاز,71,6,30,تخلیه,سنگ,شخصي
943,546,17,30,مسکونی,1,1,0,0,همكف,88,...,1,1,0,کولر_گاز,546,17,30,مسکونی,سيمان,شخصي


In [67]:
filtered_df.loc[:,'age'] = filtered_df['statuse']
filtered_df.loc[:,'statuse'] = filtered_df['doc_status']
filtered_df.loc[:,'doc_status'] = 'شخصی'
filtered_df.iloc[:,7:12]

,age,statuse,view,doc_status,north
36,13,مسکونی,30,شخصی,0
56,8,مسکونی,12,شخصی,0
174,33,تخلیه,30,شخصی,0
268,12,مسکونی,27,شخصی,0
935,14,مسکونی,30,شخصی,1
940,6,تخلیه,30,شخصی,0
943,17,مسکونی,30,شخصی,1


In [68]:
df=pd.concat([df,filtered_df])
df.shape

(979, 33)

--------------------------------------------------------------------------------------------------

check 4th None column

In [69]:
df.iloc[:, 31].value_counts()

شخصي       23
سنگ         5
تراورتن     1
سيمان       1
Name: count, dtype: int64

In [70]:
filtered_df = df[df.iloc[:, 31]=='شخصي']
df.drop(index=filtered_df.index,inplace=True)
filtered_df.iloc[:,7:12]

,age,statuse,view,doc_status,north
64,200,22,اجاره,سنگ,0
67,1200,22,مسکونی,سنگ,0
75,216,30,تخلیه,سنگ,1
97,780,17,اجاره,آجر سه سانت,0
99,400,22,تخلیه,سنگ,1
212,191,30,مسکونی,سيمان,1
243,30,3,تخلیه,سنگ,0
269,225,30,تخلیه,سنگ,1
312,225,17,تخلیه,سنگ,0
399,184,20,مسکونی,سنگ,0


In [71]:
filtered_df.loc[:,'age'] = filtered_df['statuse']
filtered_df.loc[:,'statuse'] = filtered_df['view']
filtered_df.loc[:,'view'] = filtered_df['doc_status']
filtered_df.loc[:,'doc_status'] = 'شخصي'
df=pd.concat([df,filtered_df])
df.shape

(979, 33)

In [72]:
filtered_df = df[(df.iloc[:, 32].notna()) & (df.iloc[:, 31]!='شخصي')]
df.drop(index=filtered_df.index,inplace=True)
filtered_df.iloc[:,7:]

,age,statuse,view,doc_status,north,sought,west,east,floor,area,...,parking,warehouse,balcony,equipment,nearby,None,None,None,None,None
36,13,مسکونی,30,شخصی,0,1,0,1,1,150,...,1,1,0,شوفاژ_کولر_گاز,400,13,30,مسکونی,سنگ,شخصي
56,8,مسکونی,12,شخصی,0,1,0,0,كلی,380,...,1,0,1,شوفاژ_کولر_گاز,150,8,12,مسکونی,سنگ,شخصي
174,33,تخلیه,30,شخصی,0,0,1,1,1,128,...,1,1,1,شوفاژ_کولر,1330,33,30,تخلیه,سنگ,شخصي
268,12,مسکونی,27,شخصی,0,1,0,0,4,84,...,1,1,0,کولر_گاز,300,12,27,مسکونی,سنگ,شخصي
935,14,مسکونی,30,شخصی,1,0,0,0,4,100,...,1,1,0,شوفاژ_کولر,420,14,30,مسکونی,تراورتن,شخصي
940,6,تخلیه,30,شخصی,0,1,0,0,كلی,222,...,0,1,1,کولر_گاز,71,6,30,تخلیه,سنگ,شخصي
943,17,مسکونی,30,شخصی,1,1,0,0,همكف,88,...,1,1,0,کولر_گاز,546,17,30,مسکونی,سيمان,شخصي


In [73]:
filtered_df.loc[:,'view'] = filtered_df.iloc[:,31]
df=pd.concat([df,filtered_df])
df.shape

(979, 33)

----------------------------------------------------------------------------------------------------------------------

now lets drop none named columns

In [74]:
df.reset_index(inplace=True,drop=True)

In [75]:
df=df.iloc[:,:-5]

In [76]:
df['price'] = df['price'].str.replace(' تومان', '').str.replace(',', '').astype(int)
#pandas datetime has a limited date range between 1677-09-21 and 2262-04-11
df[['year', 'month', 'day']] = df['date'].str.split('/', expand=True).astype(int)
clmn=['date','h_type','year','nearby','loc']
df.drop(columns=clmn,inplace=True) 
df.head()

,adress,num_floor,unit_per_floor,price,age,statuse,view,doc_status,north,sought,...,kitch,service,floor_covering,open,parking,warehouse,balcony,equipment,month,day
0,تهرانپارس خ 188غربی,5,2,14000000000,15,مسکونی,سنگ,شخصي,0,1,...,MDF,ايراني فرنگي,سنگ,1,1,1,1,شوفاژ_کولر_آسانسور_گاز,1,28
1,تهرانپارس بلوار پروین خ 196 شرقی,4,1,4500000000,1,تخلیه,سنگ,قولنامه اي,1,0,...,MDF,ايراني فرنگي,سراميک,1,0,1,1,پکیج,1,26
2,فرجام غربی خ حیدرخانی,5,1,10500000000,4,تخلیه,سنگ,شخصي,0,1,...,MDF,ايراني فرنگي,سراميک,1,1,1,1,کولر_آسانسور_پکیج,1,26
3,هروی گلستان,5,2,18000000000,17,مسکونی,سنگ,شخصي,1,0,...,MDF,ايراني فرنگي,سراميک,1,1,1,0,شوفاژ_کولر_آسانسور,1,31
4,هنگام چهارراه استقلال,4,2,5000000000,17,مسکونی,سنگ,شخصي,1,0,...,MDF,ايراني فرنگي,سراميک,0,1,1,0,کولر_گاز,1,26


In [77]:
df['unit_per_floor'].value_counts()

unit_per_floor
2      397
1      275
4      132
3      112
6       14
5       10
9        9
8        8
7        8
10       6
12       4
20       2
---      1
11       1
Name: count, dtype: int64

In [78]:
df['unit_per_floor']=np.where(df['unit_per_floor']=='---','2',df['unit_per_floor'])

In [79]:
df.age=np.where(df.age=='نوساز',0,df.age)

In [80]:
df[df.age=='مسکونی']

,adress,num_floor,unit_per_floor,price,age,statuse,view,doc_status,north,sought,...,kitch,service,floor_covering,open,parking,warehouse,balcony,equipment,month,day
124,هروی بین مكران جنوبی و پناهی نیا,5,2,21000000000,مسکونی,سنگ,شخصي,باز سازی شده,0,1,...,MDF,ايراني فرنگي,سراميک,1,1,1,0,کولر_آسانسور_گاز,2,3


In [81]:
df.statuse.value_counts()

statuse
مسکونی    459
تخلیه     358
اجاره     161
سنگ         1
Name: count, dtype: int64

In [82]:
df[df.statuse=='سنگ']

,adress,num_floor,unit_per_floor,price,age,statuse,view,doc_status,north,sought,...,kitch,service,floor_covering,open,parking,warehouse,balcony,equipment,month,day
124,هروی بین مكران جنوبی و پناهی نیا,5,2,21000000000,مسکونی,سنگ,شخصي,باز سازی شده,0,1,...,MDF,ايراني فرنگي,سراميک,1,1,1,0,کولر_آسانسور_گاز,2,3


In [83]:
df = df.drop(df.index[124])

In [84]:
stat_map = {
    'مسکونی': 0,
    'اجاره': 1,
    'تخلیه': 2,
}
df['statuse'] = df['statuse'].map(stat_map)
reverse_stat_map= {v: k for k, v in stat_map.items()} 

In [85]:
df.view.value_counts()

view
سنگ            851
ترکیبی          23
سيمان           23
آجر             20
آجر سه سانت     18
گرانيت          18
تراورتن         14
کنيتکس           2
رومي             2
مسکوني           1
شيشه             1
قولنامه اي       1
کامپوزيت         1
سنگ و رومي       1
شخصي             1
کلاسيک           1
Name: count, dtype: int64

In [86]:
df[(df.view=='شخصي')|(df.view=='مسکوني')|(df.view=='قولنامه اي')]

,adress,num_floor,unit_per_floor,price,age,statuse,view,doc_status,north,sought,...,kitch,service,floor_covering,open,parking,warehouse,balcony,equipment,month,day
194,تهرانپارس خ ملكی,6,2,21170000000,4,0,مسکوني,2 پاركینگ,0,1,...,چوبي,ايراني فرنگي,نا مشخص,1,1,1,1,شوفاژ_کولر_آسانسور_گاز,2,2
202,جشنواره خ زهدی,6,1,2400000000,2,0,قولنامه اي,12 متر بالكن,1,0,...,MDF,ايرانی,سراميک,1,0,0,1,کولر_آسانسور_پکیج,1,26
634,تهرانپارس خ رشید,6,2,22400000000,5,0,شخصي,,1,0,...,فرنيش,ايراني فرنگي,سنگ,1,1,1,1,شوفاژ_کولر_آسانسور_گاز,1,23


In [87]:
df.doc_status.value_counts()

doc_status
شخصي            817
قولنامه اي       77
تعاوني           34
دردست اقدام      15
مسکوني           12
منگوله دار        7
شخصی              7
مشاع              3
اوقافي            2
2 پاركینگ         1
12 متر بالكن      1
آماده محضر        1
                  1
Name: count, dtype: int64

In [88]:
df[(df.doc_status=='12 متر بالكن')|(df.doc_status=='2 پاركینگ')|(df.doc_status=='')|(df.doc_status=='آماده محضر')]

,adress,num_floor,unit_per_floor,price,age,statuse,view,doc_status,north,sought,...,kitch,service,floor_covering,open,parking,warehouse,balcony,equipment,month,day
194,تهرانپارس خ ملكی,6,2,21170000000,4,0,مسکوني,2 پاركینگ,0,1,...,چوبي,ايراني فرنگي,نا مشخص,1,1,1,1,شوفاژ_کولر_آسانسور_گاز,2,2
202,جشنواره خ زهدی,6,1,2400000000,2,0,قولنامه اي,12 متر بالكن,1,0,...,MDF,ايرانی,سراميک,1,0,0,1,کولر_آسانسور_پکیج,1,26
605,تهرانپارس شرقی بلوار پروین ك 222,4,2,3350000000,13,0,سنگ,آماده محضر,0,1,...,MDF,ايرانی,سراميک,1,0,1,0,آسانسور_پکیج,2,6
634,تهرانپارس خ رشید,6,2,22400000000,5,0,شخصي,,1,0,...,فرنيش,ايراني فرنگي,سنگ,1,1,1,1,شوفاژ_کولر_آسانسور_گاز,1,23


In [89]:
df.warehouse=np.where(df.doc_status=='12 متر بالكن','1',df.warehouse)
df.view=np.where((df.doc_status=='12 متر بالكن')|(df.doc_status=='')|(df.doc_status=='2 پاركینگ'),'سنگ',df.view)
df.doc_status=np.where((df.doc_status=='12 متر بالكن'),'قولنامه اي',df.doc_status)
df.floor_covering=np.where((df.doc_status=='2 پاركینگ'),'پارکت',df.floor_covering)
df.doc_status=np.where((df.doc_status=='شخصی')|(df.doc_status=='مسکوني')|(df.doc_status=='2 پاركینگ')|(df.doc_status=='')|(df.doc_status=='آماده محضر'),'شخصي',df.doc_status)

In [90]:
df['view'] = df['view'].replace(['کلاسيک', 'سنگ و رومي', 'کامپوزيت','شيشه','کنيتکس','رومي'], 'سایر')
view_map = {
    'سایر': 0,
    'تراورتن': 1,
    'آجر سه سانت': 2,
    'گرانيت': 3,
    'آجر': 4,
    'سيمان': 5,
    'ترکیبی': 6,
    'سنگ': 7
}
df['view'] = df['view'].map(view_map)
reverse_view_map= {v: k for k, v in view_map.items()} 

In [91]:
doc_map = {
    'شخصي': 0,
    'قولنامه اي': 1,
    'تعاوني': 2,
    'دردست اقدام': 3,
    'منگوله دار': 4,
    'مشاع': 5,
    'اوقافي': 6
}
df['doc_status'] = df['doc_status'].map(doc_map)
reverse_doc_map= {v: k for k, v in doc_map.items()} 

In [92]:
df.floor.value_counts()

floor
1          226
4          219
3          198
2          163
5           62
همكف        40
زیرهمكف     14
6           14
مختلف        9
7            7
10           6
11           5
9            4
كلی          3
8            2
16           2
دو واحد      2
15           1
سه واحد      1
Name: count, dtype: int64

In [93]:
df.floor=np.where(df.floor=='همكف',0,df.floor)
df.floor=np.where(df.floor=='زیرهمكف',-1,df.floor)

In [94]:
df.floor=np.where((df.floor=='سه واحد')|(df.floor=='دو واحد')|(df.floor=='كلی')|(df.floor=='مختلف'),'2',df.floor)

In [95]:
df.tel.value_counts()

tel
1       897
0        43
2        24
         10
3         2
ابری      1
4         1
Name: count, dtype: int64

In [96]:
df.tel=np.where(df.tel=='','0',df.tel)
df.tel=np.where(df.tel=='ابری','5',df.tel)

In [97]:
df.kitch.value_counts()

kitch
MDF          847
هایگلاس       65
چوبي          24
فلزي          15
نا مشخص       14
دلخواه         7
ندارد          3
چوبي فلزي      1
فرنيش          1
آبدارخانه      1
Name: count, dtype: int64

In [98]:
df['kitch'] = df['kitch'].replace(['چوبي فلزي', 'فرنيش', 'آبدارخانه'], 'سایر')
df['kitch'] = df['kitch'].replace(['نا مشخص', 'ندارد'], 'نامشخص')

In [99]:
#mean price of each kitchen-type as value of kitch
target_mean = df.groupby('kitch')['price'].mean().to_dict()
df['kitch'] = df['kitch'].map(target_mean)
reverse_target_map = {v: k for k, v in target_mean.items()} 

In [100]:
df.service.value_counts()

service
ايراني فرنگي    625
ايرانی          328
فرنگی            22
نامشخص            3
Name: count, dtype: int64

In [101]:
service_dict = {'ايراني فرنگي': 5,'فرنگی': 2,'ايرانی': 1,'نامشخص': 0}
reverse_service_dict={v:k for k,v in service_dict.items()}
df['service'] = df['service'].map(service_dict)

In [102]:
df.floor_covering.value_counts()

floor_covering
سراميک            639
سنگ               225
پارکت              78
لامنيت             16
موکت               10
موزائيک             4
نا مشخص             2
سراميک موزائيک      1
سنگ سراميک          1
کفپوش               1
PVC                 1
Name: count, dtype: int64

In [103]:
df['floor_covering'] = df['floor_covering'].replace(['سراميک موزائيک', 'سنگ سراميک', 'کفپوش', 'PVC'], 'سایر')

In [104]:
target_mean_fc = df.groupby('floor_covering')['price'].mean().to_dict()
df['floor_covering'] = df['floor_covering'].map(target_mean_fc)
reverse_target_map_fc = {v: k for k, v in target_mean_fc.items()} 

In [105]:
df.equipment.value_counts()

equipment
شوفاژ_کولر_آسانسور_گاز                      187
شوفاژ_کولر_گاز                              161
کولر_گاز                                    153
شوفاژ_کولر_آسانسور_پکیج_گاز                 109
کولر_آسانسور_پکیج_گاز                        79
کولر_آسانسور_پکیج                            57
شوفاژ_کولر_آسانسور                           43
کولر_آسانسور_گاز                             35
شوفاژ_کولر                                   34
شوفاژ_کولر_پکیج_گاز                          28
کولر_پکیج_گاز                                14
کولر_پکیج                                    10
آسانسور_پکیج                                  7
پکیج                                          4
آسانسور                                       4
                                              4
آسانسور_چیلر                                  4
آسانسور_چیلر_گاز                              3
شوفاژ_کولر_آسانسور_حیاط_گاز                   3
شوفاژ_کولر_آسانسور_حیاط                       2
آسانسور_گاز                   

In [106]:
core_features = ['شوفاژ', 'کولر', 'آسانسور', 'گاز', 'پکیج', 'حیاط']
other_features=['سونا','پاسیو','چیلر','فن','استخر','زیر زمین','مستخدم']
for feature in core_features:
    df[f'{feature}'] = df['equipment'].str.contains(feature).astype(int)
def count_other(x):
    items = x.split('_')
    return sum(pd.Series(items).isin(other_features))  # استفاده از isin

df['امکانات_دیگر'] = df['equipment'].apply(count_other)
df.drop(columns='equipment',inplace=True)

In [107]:
int_cols = ['num_floor', 'unit_per_floor', 'age', 'north', 'sought', 'west', 'east', 'floor', 'area', 'num_sleep', 'tel', 'open', 'parking', 'warehouse', 'balcony']
df[int_cols] = df[int_cols].apply(pd.to_numeric).astype('Int64')

In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 978 entries, 0 to 978
Data columns (total 32 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   adress          978 non-null    object 
 1   num_floor       978 non-null    Int64  
 2   unit_per_floor  978 non-null    Int64  
 3   price           978 non-null    int64  
 4   age             978 non-null    Int64  
 5   statuse         978 non-null    int64  
 6   view            978 non-null    int64  
 7   doc_status      978 non-null    int64  
 8   north           978 non-null    Int64  
 9   sought          978 non-null    Int64  
 10  west            978 non-null    Int64  
 11  east            978 non-null    Int64  
 12  floor           978 non-null    Int64  
 13  area            978 non-null    Int64  
 14  num_sleep       978 non-null    Int64  
 15  tel             978 non-null    Int64  
 16  kitch           978 non-null    float64
 17  service         978 non-null    int64  


In [109]:
districts = [
    'تهران', 'شمس', 'هروی', 'فرجام', 'حكیمیه','سیدخندان',
    'بنی هاشم', 'مجیدیه', 'جشنواره', 'هنگام', 'رسالت','دلاوران',
    'بابایی', 'عراقی','سراج','لویزان','خواجه عبد','نیرو',
]
# استخراج اولین منطقه موجود در آدرس
def extract_district(address):
    for district in districts:
        if district in address:
            return district
    return 'سایر'
df['district'] = df['adress'].apply(extract_district)

In [110]:
df['district'].value_counts()


district
تهران        352
جشنواره       98
شمس           70
فرجام         68
هروی          59
هنگام         54
حكیمیه        42
بنی هاشم      33
بابایی        28
سراج          28
لویزان        27
خواجه عبد     24
مجیدیه        21
سایر          21
رسالت         15
عراقی         15
دلاوران       12
نیرو           8
سیدخندان       3
Name: count, dtype: int64

In [111]:
district_map = {
    'نیرو': 0,
    'دلاوران': 1,
    'عراقی': 2,
    'رسالت': 3,
    'لویزان': 4,
    'سراج': 5,
    'سایر': 6,
    'بنی هاشم': 7,'هنگام': 8,'شمس': 9,
    'مجیدیه': 10,
    'خواجه عبد': 11,
    'جشنواره': 12,
    'فرجام': 13,
    'بابایی': 14,
    'حكیمیه': 15,
    'هروی': 16,
    'سیدخندان': 17,
    'تهران': 18,
}
df['district'] = df['district'].map(district_map)
reverse_district_map= {v: k for k, v in district_map.items()} 

In [112]:
def combine_directions(row):
    directions = []
    if row['north']:
        directions.append('north')
    if row['east']:
        directions.append('east')
    if row['west']:
        directions.append('west')
    if row['sought']:
        directions.append('south')
    return ','.join(directions) if directions else '-1'

df['direction'] = df.apply(combine_directions, axis=1)

In [113]:
dir_map = {
    'north': 4,
    'north,east': 3.5,
    'east': 3,
    'south,east': 2.5,
    'south': 2,
    'west': 1,
    'north,west': 1.5,
    'unknown': 0
}
df['direction'] = df['direction'].map(dir_map)

In [114]:
df.direction=np.where(df.direction.isna(),-1,df.direction)

In [115]:
df = df.drop(columns=['north', 'east', 'west', 'sought'])

In [116]:
df_n=df.iloc[:,1:].copy()

In [128]:
df_n.head(20).iloc[:,15:]

,parking,warehouse,balcony,month,day,شوفاژ,کولر,آسانسور,گاز,پکیج,حیاط,امکانات_دیگر,district,direction
0,1,1,1,1,28,1,1,1,1,0,0,0,18,2.0
1,0,1,1,1,26,0,0,0,0,1,0,0,18,4.0
2,1,1,1,1,26,0,1,1,0,1,0,0,13,2.0
3,1,1,0,1,31,1,1,1,0,0,0,0,16,4.0
4,1,1,0,1,26,0,1,0,1,0,0,0,8,4.0
5,0,1,0,2,3,1,1,0,1,0,0,0,18,4.0
6,1,1,0,1,27,1,1,1,1,0,0,0,12,2.0
7,1,1,1,1,23,0,1,0,0,1,0,0,12,4.0
8,1,1,0,2,1,1,1,1,1,0,0,0,18,4.0
9,1,1,1,1,31,0,1,0,1,1,0,0,12,3.0


In [117]:
df_n.describe().iloc[:,:15]

,num_floor,unit_per_floor,price,age,statuse,view,doc_status,floor,area,num_sleep,tel,kitch,service,floor_covering,open
count,978.0,978.0,9.780000e+02,978.0,978.000000,978.000000,978.000000,978.0,978.0,978.0,978.0,9.780000e+02,978.000000,9.780000e+02,978.0
mean,4.803681,2.481595,1.004240e+10,13.301636,0.896728,6.559305,0.251534,2.755624,92.102249,1.919223,0.981595,1.004240e+10,3.575665,1.004240e+10,0.947853
std,2.1357,1.877565,7.350777e+09,9.234798,0.908602,1.334167,0.747649,1.920262,37.741427,0.729983,0.335038,1.891162e+09,1.902673,2.620581e+09,0.222438
min,2.0,1.0,1.300000e+09,0.0,0.000000,0.000000,0.000000,-1.0,30.0,0.0,0.0,6.463867e+09,0.000000,3.060000e+09,0.0
25%,4.0,1.0,5.350000e+09,5.0,0.000000,7.000000,0.000000,1.0,67.0,1.0,1.0,9.591502e+09,1.000000,8.571502e+09,1.0
50%,4.0,2.0,8.000000e+09,14.5,1.000000,7.000000,0.000000,3.0,85.0,2.0,1.0,9.591502e+09,5.000000,8.571502e+09,1.0
75%,5.0,3.0,1.260000e+10,20.0,2.000000,7.000000,0.000000,4.0,107.0,2.0,1.0,9.591502e+09,5.000000,9.445692e+09,1.0
max,24.0,20.0,8.896000e+10,33.0,2.000000,7.000000,6.000000,16.0,510.0,9.0,5.0,3.080000e+10,5.000000,1.472797e+10,1.0


In [118]:
X = df_n.drop(columns=['price'])
y = df_n['price']

In [184]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
x_train.shape

(782, 28)

In [185]:
sc_x=StandardScaler()
x_train_sc=sc_x.fit_transform(x_train)
x_test_sc=sc_x.transform(x_test)
sc_y=StandardScaler()
y_train_sc=sc_y.fit_transform(y_train.to_frame())
y_test_sc=sc_y.transform(y_test.to_frame())

In [186]:
sgdr=SGDRegressor(random_state=42,)
sgdr.fit(x_train_sc,y_train_sc)
y_pred=sgdr.predict(x_train_sc)
y_pred_test=sgdr.predict(x_test_sc)
r2_score(y_train_sc,y_pred),r2_score(y_test_sc,y_pred_test)

c:\Users\User\Desktop\MLtasks\env\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(0.7815185427559912, 0.6501559766402735)

In [187]:
ransac=RANSACRegressor(
    estimator=LinearRegression(),
    min_samples=0.1,  # حداقل 10% از داده‌ها برای تخمین مدل
    max_trials=100,  # حداکثر تعداد تکرار
    random_state=42,
)
ransac.fit(x_train_sc,y_train_sc)
y_pred_r=ransac.predict(x_train_sc)
y_pred_test_r=ransac.predict(x_test_sc)
r2_score(y_train_sc,y_pred_r),r2_score(y_test_sc,y_pred_test_r)

(0.7120042094654064, 0.7686879276824434)